In [14]:
pip install sacremoses


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 1.4 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
dataset_path = "merged_dataset.csv"
df = pd.read_csv(dataset_path)

# Ensure dataset has necessary columns
assert "ayah" in df.columns and "translation_with_the_updated_meaning" in df.columns, \
    "Dataset must have 'ayah' and 'translation_with_the_updated_meaning' columns"

# Load dataset
dataset_path = "merged_dataset.csv"
df = pd.read_csv(dataset_path)

# Ensure dataset has necessary columns
assert "ayah" in df.columns and "translation_with_the_updated_meaning" in df.columns, \
    "Dataset must have 'ayah' and 'translation_with_the_updated_meaning' columns"

# Format dataset for training and evaluation
def format_prompt(example):
    return {
        "input_text": example['ayah'],  # Arabic Ayah as the input
        "target_text": example['translation_with_the_updated_meaning']  # English translation as the output
    }

# Convert to Hugging Face dataset format
full_dataset = Dataset.from_pandas(df).map(format_prompt, remove_columns=["ayah", "translation_with_the_updated_meaning"])

# Use full dataset for both training and evaluation
datasets = DatasetDict({
    "train": full_dataset,
    "eval": full_dataset
})

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [12]:
print(datasets["train"][0])
 

{'root': 'ضرب', 'the_new_meaning': 'to separate', 'input_text': 'فأوحتنا إلى مُوسى أن اضرب تفضالا البخرة فاتقلق فكان كلّ يؤثي كاظفود الغظيم', 'target_text': "Then We inspired Moses, 'separate the sea with your staff,' so it parted, and each part became like a huge mountain."}


In [17]:
# Load MarianMT model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenize the data
def tokenize_function(examples):
    inputs = tokenizer(
        examples["input_text"], padding="max_length", truncation=True, max_length=128
    )
    targets = tokenizer(
        examples["target_text"], padding="max_length", truncation=True, max_length=128
    )

    # Set padding tokens to -100 for loss calculation
    inputs["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels]
        for labels in targets["input_ids"]
    ]
    
    return inputs


tokenized_datasets = datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [18]:
# Print dataset statistics
df["input_length"] = df["ayah"].apply(lambda x: len(tokenizer(x)["input_ids"]))
df["target_length"] = df["translation_with_the_updated_meaning"].apply(lambda x: len(tokenizer(x)["input_ids"]))
print(df[["input_length", "target_length"]].describe())


       input_length  target_length
count      8.000000        8.00000
mean     107.625000       98.62500
std       70.920353       55.40742
min       23.000000       19.00000
25%       67.750000       61.00000
50%       93.500000       95.00000
75%      140.750000      140.25000
max      227.000000      183.00000


In [26]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=30,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch"
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"]
)

# Train the model
trainer.train()

# Save the model after training
trainer.save_model("./trained_model")
tokenizer.save_pretrained("./trained_model")

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.667207
2,No log,1.435050
3,No log,1.268045
4,No log,1.109165
5,No log,0.962400
6,No log,0.827012
7,No log,0.713510
8,No log,0.623215
9,No log,0.552748
10,1.396900,0.496832


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/added_tokens.json')

In [27]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

model_path = "./trained_model"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

translator = pipeline("translation", model=model, tokenizer=tokenizer)

arabic_text = "فأوحتنا إلى مُوسى أن اضرب تفضالا البخرة فاتقلق فكان كلّ يؤثي كاظفود الغظيم"
translation = translator(arabic_text)
print(translation[0]["translation_text"])



Device set to use cuda:0


Then We inspired Moses, 'separate the sea with your staff,' so it parted, and each part became like a huge mountain.


Translated Text: 
